# Collision checking tutorial 💥

This notebook shows how to:
* Set up collision checking for joint configurations
* Apply collision checking to joint and TCP paths

## 1. Creating a `SceneGraphCollisionChecker` 🛡️
Drake provides a very convenient class called [`SceneGraphCollisionChecker`](https://drake.mit.edu/doxygen_cxx/group__planning__collision__checker.html) that checks for collisions between the robot's links itself and its environment.
After you set it up with a `RobotDiagram`, you can check for collisions with:
```python
collision_checker.CheckConfigCollisionFree(q)
```
Where `q` are all the joint positions of the `MultibodyPlant`.

In the previous notebooks we showed how to quickly create a `RobotDiagram` with a UR5e and Robotiq 2F-85 gripper.
You might have noticed that gripper introduces 6 degrees of freedom.
However, in this notebook we will, for simplicity, make the gripper static and do collision checks with the gripper in a fixed open position.

In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder
from airo_drake import add_floor, add_meshcat, finish_build, X_URTOOL0_ROBOTIQ, X_URBASE_ROSBASE
from airo_drake import SingleArmScene
import airo_models

robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)
add_floor(robot_diagram_builder)

plant = robot_diagram_builder.plant()
parser = robot_diagram_builder.parser()
parser.SetAutoRenaming(True)

# Load URDF files
arm_name = "ur5e"
gripper_name = "robotiq_2f_85"
arm_urdf_path = airo_models.get_urdf_path(arm_name)
gripper_urdf_path = airo_models.get_urdf_path(gripper_name)

# Make the gripper static
gripper_urdf = airo_models.urdf.read_urdf(gripper_urdf_path)
airo_models.urdf.replace_value(gripper_urdf, "@type", "revolute", "fixed")
airo_models.urdf.delete_key(gripper_urdf, "mimic")
airo_models.urdf.delete_key(gripper_urdf, "transmission")
gripper_static_urdf_path = airo_models.urdf.write_urdf_to_tempfile(
    gripper_urdf, gripper_urdf_path, prefix=f"{gripper_name}_static_"
)

# Use static gripper
arm_index = parser.AddModels(arm_urdf_path)[0]
gripper_index = parser.AddModels(gripper_static_urdf_path)[0]

# Weld some frames together
world_frame = plant.world_frame()
arm_frame = plant.GetFrameByName("base_link", arm_index)
arm_tool_frame = plant.GetFrameByName("tool0", arm_index)
gripper_frame = plant.GetFrameByName("base_link", gripper_index)

arm_rigid_transform = X_URBASE_ROSBASE
gripper_rigid_transform = X_URTOOL0_ROBOTIQ

plant.WeldFrames(world_frame, arm_frame, arm_rigid_transform)
plant.WeldFrames(arm_tool_frame, gripper_frame, gripper_rigid_transform)

robot_diagram, context = finish_build(robot_diagram_builder, meshcat)
del robot_diagram_builder  # no longer needed

scene = SingleArmScene(robot_diagram, arm_index, gripper_index, meshcat)
scene

In [ ]:
plant.num_positions(), plant.num_positions(arm_index), plant.num_positions(gripper_index)

In [ ]:
from pydrake.planning import SceneGraphCollisionChecker

collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[scene.arm_index, scene.gripper_index],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

## 2. Checking joint configurations 🤖

Lets test some joint configurations for collisions!

First, the default configuration. You can see in Meshcat that the robot is just barely not colliding with the table.

In [ ]:
collision_checker.CheckConfigCollisionFree(np.zeros(6))

UR robots are always in self-collsion when elbow joint is at 180 degrees:

In [ ]:
joints_self_collision = np.deg2rad([0, 0, 180, 0, 0, 0])
collision_checker.CheckConfigCollisionFree(joints_self_collision)

Lets double-check that visually:

In [ ]:
plant = scene.robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

plant.SetPositions(plant_context, scene.arm_index, joints_self_collision)
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization

We can make the robot collide with the table by make the shoulder angle slightly larger

In [ ]:
joints_table_collision = np.deg2rad([0, 5, 0, 0, 0, 0])
collision_checker.CheckConfigCollisionFree(joints_table_collision)

In [ ]:
plant.SetPositions(plant_context, scene.arm_index, joints_table_collision)
scene.robot_diagram.ForcedPublish(context)

## Checking joint paths 🛤️

In [ ]:
start_joints = np.deg2rad([0, -90, 90, -90, -90, 0])

print(collision_checker.CheckConfigCollisionFree(start_joints))

plant.SetPositions(plant_context, scene.arm_index, start_joints)
scene.robot_diagram.ForcedPublish(context)

In [ ]:
goal_joints = np.deg2rad([0, -90, 90, 180, -110, 0])

print(collision_checker.CheckConfigCollisionFree(goal_joints))

plant.SetPositions(plant_context, scene.arm_index, goal_joints)
scene.robot_diagram.ForcedPublish(context)

In [ ]:
from airo_drake import animate_joint_configurations

joint_path = np.linspace(start_joints, goal_joints, 20)
animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, joint_path, context=context)

In [ ]:
def path_collisions_as_emojis(is_collision_free: list[bool]):
    """Displays an emoji-based visualization of a path's collisions.

    Example output: "✅✅💥✅✅✅💥✅✅✅✅"

    Args:
        is_collision_free: A list of booleans, where True indicates no collision.

    Returns:
        A string of emojis representing the collision status of the path.
    """
    emojis = ["✅" if is_free else "💥" for is_free in is_collision_free]
    emoji_str = "".join(emojis)
    return emoji_str

print(collision_checker.CheckConfigsCollisionFree(joint_path))
path_collisions_as_emojis(collision_checker.CheckConfigsCollisionFree(joint_path))

## Checking TCP paths 📏

For the cloth competition, I've had the problem that my `pregrasp_pose` pose is collision-free, but movely linearly to the `grasp_pose` make the robot elbow collide with the table.

Below is a recreation of this scenario, and a demonstration of how checking the TCP path can help us avoid this problem.

In [ ]:
from airo_drake import visualize_frame


# grasp_location = np.array([0.2, 0.5, 0.5]) # This moves the robot through a singularlity for some start configurations
grasp_location = np.array([0.2, 0.4, 0.55])

gripper_forward_direction = np.array([1, 0, 0])

Z = gripper_forward_direction / np.linalg.norm(gripper_forward_direction)
Y = np.array([0, 0, -1])  # 0, 0, 1 is also an option
X = np.cross(Y, Z)

grasp_orientation = np.column_stack([X, Y, Z])
grasp_pose = np.identity(4)
grasp_pose[0:3, 0:3] = grasp_orientation
grasp_pose[0:3, 3] = grasp_location

pregrasp_pose = grasp_pose.copy()
pregrasp_pose[0:3, 3] -= 0.4 * gripper_forward_direction

visualize_frame(scene.meshcat, pregrasp_pose, "pregrasp_pose")
visualize_frame(scene.meshcat, grasp_pose, "grasp_pose")

In [ ]:
from ur_analytic_ik import ur5e

tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175 # 175 mm in z

start_configurations = ur5e.inverse_kinematics_with_tcp(pregrasp_pose, tcp_transform)
goal_configurations = ur5e.inverse_kinematics_with_tcp(grasp_pose, tcp_transform)

start_configurations = np.array(start_configurations).squeeze()
goal_configurations = np.array(goal_configurations).squeeze()

len(start_configurations), len(goal_configurations)

In [ ]:
print(path_collisions_as_emojis(collision_checker.CheckConfigsCollisionFree(start_configurations)))
print(path_collisions_as_emojis(collision_checker.CheckConfigsCollisionFree(goal_configurations)))

In [ ]:
animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, start_configurations, duration=len(start_configurations))

In [ ]:
animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, goal_configurations, duration=len(goal_configurations))

In [ ]:
from airo_typing import HomogeneousMatrixType


def interpolate_linearly(a, b, t):
    return a + t * (b - a)


def interpolate_pose_path_positions(pose_a: HomogeneousMatrixType, pose_b: HomogeneousMatrixType, n: int):
    pose_path = []

    orientation = pose_a[0:3, 0:3]
    for i in np.linspace(0, 1, n):
        pose_interpolated = np.identity(4)
        position = interpolate_linearly(pose_a[0:3, 3], pose_b[0:3, 3], i)
        pose_interpolated[0:3, 0:3] = orientation
        pose_interpolated[0:3, 3] = position
        pose_path.append(pose_interpolated)
    return pose_path


tcp_path = interpolate_pose_path_positions(pregrasp_pose, grasp_pose, n=20)

for i, tcp_pose in enumerate(tcp_path):
    visualize_frame(scene.meshcat, tcp_pose, f"tcp_path/pose_{i}", length=0.05, opacity=0.1)


path_joints_solutions = []

for tcp_pose in tcp_path:
    joint_solutions = ur5e.inverse_kinematics_with_tcp(tcp_pose, tcp_transform)
    path_joints_solutions.append(np.array(joint_solutions).squeeze())

    
path_joints_solutions = np.array(path_joints_solutions) # note this is not always possible, because the number of solutions can change
path_joints_solutions.shape

In [ ]:
from airo_drake import create_paths_from_closest_solutions

paths = create_paths_from_closest_solutions(path_joints_solutions)
paths = np.array(paths)
paths.shape

In [ ]:
for path in paths:
    print(path_collisions_as_emojis(collision_checker.CheckConfigsCollisionFree(path)))

# for row in np.swapaxes(paths, 0, 1):
#     print(path_collisions_as_emojis(collision_checker.CheckConfigsCollisionFree(row)))
# np.allclose(np.swapaxes(paths, 0, 1), path_joints_solutions)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from airo_drake import calculate_joint_path_outlier_threshold

path_distances = [np.linalg.norm(np.diff(path, axis=0), axis=1) for path in paths]
thresholds = [calculate_joint_path_outlier_threshold(distances) for distances in path_distances]

colors = matplotlib.colormaps["tab10"].colors[:len(paths)]
plt.figure(figsize=(20, 8))
plt.title("Distances between the consecutive configurations in the paths")
for distances, threshold, color in zip(path_distances, thresholds, colors):
    plt.plot(distances, marker="o", color=color)
    plt.axhline(y=threshold, color=color, linestyle='--')
plt.legend([f"path {i}" for i in range(len(paths))] + [f"path {i} upper bound" for i in range(len(paths))])
plt.show()

In [ ]:
from airo_drake import joint_path_has_large_jumps

for i, path in enumerate(paths):
    has_jumps = joint_path_has_large_jumps(path)

    distances = np.linalg.norm(np.diff(path, axis=0), axis=1)
    threshold = calculate_joint_path_outlier_threshold(distances)
    jumps = np.where(distances > threshold)[0]

    emoji = "🦘" if has_jumps else "✅"
    print(f"path {i}: {emoji} {jumps}")



In [ ]:
paths_without_jumps = [path for path in paths if not joint_path_has_large_jumps(path)]

for path in paths_without_jumps:
    print(path_collisions_as_emojis(collision_checker.CheckConfigsCollisionFree(path)))

In [ ]:
paths_with_collisions = [path for path in paths_without_jumps if not np.all(collision_checker.CheckConfigsCollisionFree(path))]

for path in paths_with_collisions:
    print(path_collisions_as_emojis(collision_checker.CheckConfigsCollisionFree(path)))

animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, paths_with_collisions[0])

In [ ]:
paths_without_collisions = [path for path in paths_without_jumps if np.all(collision_checker.CheckConfigsCollisionFree(path))]

for path in paths_without_collisions:
    print(path_collisions_as_emojis(collision_checker.CheckConfigsCollisionFree(path)))

animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, paths_without_collisions[0])